### N Gram Analysis According To Fourgram

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index      

In [ ]:
ngram_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Data/1-N Gram Data Prepare"

ngram_used_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Data/N Gram Used"


path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/2-N Gram Analysis According To Fourgram"

Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
def strip(df,column_list):
    '''
    strip(df,column_list) df is dataframe and column_list are selected columns \n
    strip(df_twogram_used,df_twogram_used.columns)
    '''
    for i in column_list:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [ ]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed  
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [ ]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=False)]  # na=True failed    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Ngram Read

In [ ]:
df_twogram_used = pd.read_excel(f"{ngram_used_data_path}/{lang_folder.capitalize()}_Twogram_Used.xlsx")
df_twogram_used = strip(df_twogram_used,df_twogram_used.columns)
df_twogram_used

In [ ]:
df_threegram_used = pd.read_excel(f"{ngram_used_data_path}/{lang_folder.capitalize()}_Threegram_Used.xlsx")
df_threegram_used = strip(df_threegram_used, df_threegram_used.columns)
df_threegram_used

In [ ]:
twogram_used_list = list(set(df_twogram_used["twogram"]))
threegram_used_list = list(set(df_threegram_used["threegram"]))

In [ ]:
df_twogram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Twogram_Selected.csv")
df_twogram_remove_used = df_twogram[~df_twogram["twogram"].isin(twogram_used_list)]
df_twogram_remove_used

In [ ]:
df_threegram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Threegram_Selected.csv")
df_threegram_remove_used = df_threegram[~df_threegram["threegram"].isin(threegram_used_list)]
df_threegram_remove_used

In [ ]:
df_fourgram = pd.read_csv(f"{ngram_data_path}/{lang_folder.capitalize()}_{word_end}_Word_Fourgram_Selected.csv")
df_fourgram

In [ ]:
df_fourgram_selected_for_course = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/\
Data/Manuel Select/Selected 4grams.xlsx")
df_fourgram_selected_for_course

In [ ]:
twogram_list = df_twogram_remove_used["twogram"].to_list()  # without used
threegram_list = df_threegram_remove_used["threegram"].to_list()  # without used
fourgram_list = df_fourgram["fourgram"].to_list()
fourgram_course_list = df_fourgram_selected_for_course["fourgram"].to_list()

#### Given Previous Course

##### Twogram Used In Threegram Used In Fourgram (1A)

In [ ]:
# parameter
source_wordgroup_list = threegram_used_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_threegram_used_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_threegram_used_in_selected_four

In [ ]:
df_threegram_used_in_selected_four2 = df_threegram_used_in_selected_four[["threegram"]]
df_threegram_used_in_selected_four2.drop_duplicates(inplace=True)
df_threegram_used_in_selected_four2.reset_index(drop=True, inplace=True)
df_threegram_used_in_selected_four2

In [ ]:
# parameter
source_wordgroup_list = twogram_used_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_used_in_selected_four2

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_twogram_used_in_threegram_used_in_selected_four2 = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_twogram_used_in_threegram_used_in_selected_four2

In [ ]:
df_twogram_used_in_three_used_in_selected_four = pd.merge(df_twogram_used_in_threegram_used_in_selected_four2, df_threegram_used_in_selected_four, how="inner", on="threegram")
df_twogram_used_in_three_used_in_selected_four.drop_duplicates(inplace=True)
df_twogram_used_in_three_used_in_selected_four

In [ ]:
#df_twogram_used_in_three_used_in_selected_four.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_In_Threegram_Used_In_Selected_Fourgram_Result.xlsx", index=False)  # (1A)

##### Twogram Used In Selected Fourgram (2A)

In [ ]:
# parameter
source_wordgroup_list = twogram_used_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_used_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_used_in_selected_four

In [ ]:
df_two_used_in_selected_four2 = df_two_used_in_selected_four[~df_two_used_in_selected_four["twogram"].isin(list(set(df_twogram_used_in_three_used_in_selected_four["twogram"])))]
df_two_used_in_selected_four2

In [ ]:
#df_two_used_in_selected_four2.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_In_Selected_Fourgram_Result.xlsx", index=False)

##### Threegram Used In Selected Fourgram (4A)

In [ ]:
# parameter
source_wordgroup_list = threegram_used_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_used_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_used_in_selected_four

In [ ]:
df_three_used_in_selected_four2 = df_three_used_in_selected_four[~df_three_used_in_selected_four["threegram"].isin(list(set(df_twogram_used_in_three_used_in_selected_four["threegram"])))]
df_three_used_in_selected_four2

In [ ]:
#df_three_used_in_selected_four2.to_excel(f"{lang_folder.capitalize()}_Threegram_Used_In_Selected_Fourgram_Result.xlsx", index=False)

##### Twogram Used In Threegram Used (3A)

In [ ]:
# parameter
source_wordgroup_list = twogram_used_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_used

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_used_in_three_used = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_used_in_three_used

In [ ]:
three_in_selected_four_list = list(set(df_twogram_used_in_three_used_in_selected_four["threegram"]))

In [ ]:
df_two_used_in_three_used_not_in_four = df_two_used_in_three_used[~df_two_used_in_three_used["threegram"].isin(three_in_selected_four_list)]
df_two_used_in_three_used_not_in_four

In [ ]:
#df_two_used_in_three_used_not_in_four.to_excel(f"{lang_folder.capitalize()}_Twogram_Used_In_Threegram_Used_Result.xlsx", index=False)  # 3A 

#### Give Reveal Course

##### Twogram In Threegram In Selected Fourgram (1B)

In [143]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [144]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_threegram_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_threegram_in_selected_four

,threegram,fourgram
0,her şey yolunda,her şey yolunda mı
1,sana bir şey,sana bir şey söyleyeyim
2,böyle bir şey,hiç böyle bir şey
3,böyle bir şey,böyle bir şey olmayacak
4,ne işin var,burada ne işin var
...,...,...
142,sakin ol tamam,sakin ol tamam mı
143,istiyor musun istemiyor,istiyor musun istemiyor musun
144,şey söyleyeyim mi,bir şey söyleyeyim mi
145,olur ne olmaz,ne olur ne olmaz


In [145]:
df_threegram_in_selected_four2 = df_threegram_in_selected_four[["threegram"]]
df_threegram_in_selected_four2.drop_duplicates(inplace=True)
df_threegram_in_selected_four2.reset_index(drop=True, inplace=True)
df_threegram_in_selected_four2

<ipython-input-145-b6cd2c5833f9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_threegram_in_selected_four2.drop_duplicates(inplace=True)


,threegram
0,her şey yolunda
1,sana bir şey
2,böyle bir şey
3,ne işin var
4,öyle bir şey
...,...
128,sakin ol tamam
129,istiyor musun istemiyor
130,şey söyleyeyim mi
131,olur ne olmaz


In [146]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_in_selected_four2

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [147]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_twogram_in_threegram_in_selected_four2 = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_twogram_in_threegram_in_selected_four2

,twogram,threegram
0,ne demek,da ne demek
1,ne demek,ne demek oluyor
2,ne demek,ne demek biliyor
3,ne zaman,son ne zaman
4,da ne,da ne demek
...,...,...
206,şey buldun,bir şey buldun
207,şey buldun,şey buldun mu
208,öyle ya,öyle ya da
209,şey kontrol,her şey kontrol


In [148]:
df_twogram_in_three_in_selected_four = pd.merge(df_twogram_in_threegram_in_selected_four2, df_threegram_in_selected_four, how="inner", on="threegram")
df_twogram_in_three_in_selected_four.drop_duplicates(inplace=True)
df_twogram_in_three_in_selected_four

,twogram,threegram,fourgram
0,ne demek,da ne demek,bu da ne demek
1,ne demek,da ne demek,da ne demek oluyor
2,da ne,da ne demek,bu da ne demek
3,da ne,da ne demek,da ne demek oluyor
4,ne demek,ne demek oluyor,da ne demek oluyor
...,...,...,...
227,mi hayır,evet mi hayır,evet mi hayır mı
228,yapıyorsun değil,yapıyorsun değil mi,şaka yapıyorsun değil mi
229,şey buldun,bir şey buldun,bir şey buldun mu
230,öyle ya,öyle ya da,öyle ya da böyle


In [ ]:
#df_twogram_in_three_in_selected_four.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_In_Selected_Fourgram_Result.xlsx", index=False)

##### Twogram In Selected Fourgram (2B)

In [149]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [150]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_selected_four

,twogram,fourgram
0,ne demek,bu da ne demek
1,ne demek,da ne demek oluyor
2,ne demek,bu ne demek oluyor
3,ne demek,ne demek biliyor musun
4,ne demek,ne demek oluyor bu
...,...,...
209,mi hayır,evet mi hayır mı
210,yapıyorsun değil,şaka yapıyorsun değil mi
211,şey buldun,bir şey buldun mu
212,öyle ya,öyle ya da böyle


In [ ]:
#df_two_in_selected_four_list = df_two_in_selected_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
#df_two_in_selected_four_list

In [151]:
df_two_in_selected_four2 = df_two_in_selected_four[~df_two_in_selected_four["twogram"].isin(list(set(df_twogram_in_three_in_selected_four["twogram"])))]
df_two_in_selected_four2

,twogram,fourgram
10,başka bir,başka bir şey var
11,başka bir,başka bir şey yok
12,başka bir,başka bir şey değil
13,başka bir,başka bir şey daha
30,şey var,bir şey var mı
31,şey var,gereken bir şey var
32,şey var,başka bir şey var
33,şey var,istediğim bir şey var
34,şey var,istediğin bir şey var
37,de öyle,evet ben de öyle


In [ ]:
#df_two_in_selected_four2.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Selected_Fourgram_Result.xlsx", index=False)

##### Threegram In Selected Fourgram (4B)

In [152]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_selected_for_course

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [153]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_selected_four = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_three_in_selected_four

,threegram,fourgram
0,her şey yolunda,her şey yolunda mı
1,sana bir şey,sana bir şey söyleyeyim
2,böyle bir şey,hiç böyle bir şey
3,böyle bir şey,böyle bir şey olmayacak
4,ne işin var,burada ne işin var
...,...,...
142,sakin ol tamam,sakin ol tamam mı
143,istiyor musun istemiyor,istiyor musun istemiyor musun
144,şey söyleyeyim mi,bir şey söyleyeyim mi
145,olur ne olmaz,ne olur ne olmaz


In [ ]:
#df_three_in_selected_four_list = df_three_in_selected_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
#df_three_in_selected_four_list

In [156]:
df_three_in_selected_four2 = df_three_in_selected_four[~df_three_in_selected_four["threegram"].isin(list(set(df_twogram_in_three_in_selected_four["threegram"])))]
df_three_in_selected_four2

,threegram,fourgram


In [ ]:
#df_three_in_selected_four2.to_excel(f"{lang_folder.capitalize()}_Threegram_In_Selected_Fourgram_Result.xlsx", index=False)

##### Twogram In Threegram (3B)

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram

# function
#searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
#searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=False)]  # na=True failed
        for result in word_in_word_cluster[f"{target_column}"]:
            #searchlist.append(wordgroup)
            #resultlist.append(result)
            resultlist.append([wordgroup,result])
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [ ]:
#search_list = list(searchlist)
result_list = list(resultlist)
#df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_three = pd.DataFrame(result_list, columns=[f"{search_column_name}",f"{target_column}"])
df_two_in_three

In [ ]:
df_two_in_threegram_list = df_two_in_three.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
df_two_in_threegram_list

In [ ]:
#df_two_in_threegram_list.to_excel(f"{lang_folder.capitalize()}_Twogram_In_Threegram_Result.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Text_Selection_Result*")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

In [ ]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass